# what is optimization of hidden variable? Why do we need it? -- start from simple gaussian

## benefit of modeling hidden

The answer is simple: compared with fully connected model of observable, model with hidden is simpler, wich less parameters.

## estimate hidden distribution: why and how

Think of a hidden variable model, h --> x.

$$p(x|\pi, \theta) = \sum_i \pi_i p(x|\theta_{h_i})$$

Now we get a sample from data, mark as x_0. You have no idea the corresponding hidden. That is, although we have this model, we have no idea on where x is from! This is not good.(why?)

How to solve this problem? To know where x is from, we need p(h|x).

We then realise that this can be calculated by Bayes rule, $$p(h|x) = \frac{p(h,x)}{p(x)} = \frac{p(x|h_i)p(h_i)}{\sum_i p(x|h_i)p(h_i)}$$

Here $p(h_i)$ is the prior of gaussian distribution, as \mu_i and \Sigma_i\in R^{dxd}. x\in R^d. i\in {1,...,k}, represent multiple gaussian distributions are added to get the final distribution.

Then we realize something: if we know p(x|h), which is the variable $\pi$ of model distribution, and the prior, which is the mean $\mu$ and variance $\Sigma$ of gaussian distribution, the posterior can be estimated by

$$\pi_i = p(\theta_i|x)/\sum_i p(\theta_i|x)$$ 
-> the higher the possibility of p(h_i|x), the more likely x is from distribution of h.

$$\mu_i = $$

What is the benefit? We can update prior by new estimated posterior, and get the new p(h|x). Continue this, we have the converge result.

Full process:
build model $p(x|\pi,\mu) = $ x conitionals on several gaussian (multi-dim gaussian with different weighting parameter)
get data sample $data\in x^{k,n}$
get p(h|x_j) = p(x|\pi,\mu) \pi\mu is prior. => possibility of x from certain gaussian compoment
average on all x to get possibility of data from certain gaussian. => update pi of each gaussian
average on all data to get the vec mean, each gaussian get the part of its distribution mean_i => update \mu of each gaussian
the new mean and old mean on each data averaged, deduce and square to get variance => update \Sigma of each gaussian


The following is an example of learning Bayes Neural network.

Problem set up: candy from bags

you have two bags, each

The following is an example of this on SwissRoll dataset.

# reference

http://vision.psych.umn.edu/users/schrater/schrater_lab/courses/AI2/em1.pdf

http://www.ai.mit.edu/courses/6.825/fall02/pdf/6.825-lecture-18.pdf


In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

def initialize_parameters(k, d):
    """
    Initialize GMM parameters.
    k: Number of components
    d: Dimension of the data
    """
    np.random.seed(42)
    pi = np.ones(k) / k  # Equal mixture weights
    mu = np.random.rand(k, d) * 10  # Random means in the range [0, 10)
    sigma = np.array([np.eye(d) for _ in range(k)])  # Identity covariance matrices
    return pi, mu, sigma

def e_step(data, pi, mu, sigma, k):
    """
    E-step: Compute the posterior probabilities p(h|x_i)
    """
    n = data.shape[0]
    gamma = np.zeros((n, k))
    for h in range(k):
        # Multivariate normal probability density
        gamma[:, h] = pi[h] * multivariate_normal.pdf(data, mean=mu[h], cov=sigma[h])
    gamma /= gamma.sum(axis=1, keepdims=True)  # Normalize across components
    return gamma

def m_step(data, gamma, k):
    """
    M-step: Update the parameters {π_h, μ_h, Σ_h} based on gamma
    """
    n, d = data.shape
    n_h = gamma.sum(axis=0)  # Effective number of points in each component
    pi = n_h / n  # Update mixture weights
    mu = np.dot(gamma.T, data) / n_h[:, np.newaxis]  # Update means
    sigma = np.zeros((k, d, d))
    for h in range(k):
        diff = data - mu[h]  # x_i - μ_h
        sigma[h] = np.dot(gamma[:, h] * diff.T, diff) / n_h[h]  # Update covariances
    return pi, mu, sigma

def log_likelihood(data, pi, mu, sigma, k):
    """
    Compute the log-likelihood of the data given the model parameters
    """
    n = data.shape[0]
    likelihood = 0
    for h in range(k):
        likelihood += pi[h] * multivariate_normal.pdf(data, mean=mu[h], cov=sigma[h])
    return np.sum(np.log(likelihood))

def em_algorithm(data, k, max_iter=100, tol=1e-4):
    """
    EM algorithm for Gaussian Mixture Models
    """
    n, d = data.shape
    pi, mu, sigma = initialize_parameters(k, d)  # Initialize parameters
    prev_likelihood = None

    for iteration in range(max_iter):
        # E-step: Compute responsibilities
        gamma = e_step(data, pi, mu, sigma, k)

        # M-step: Update parameters
        pi, mu, sigma = m_step(data, gamma, k)

        # Compute log-likelihood
        likelihood = log_likelihood(data, pi, mu, sigma, k)
        print(f"Iteration {iteration + 1}, Log-Likelihood: {likelihood}")

        # Check for convergence
        if prev_likelihood is not None and abs(likelihood - prev_likelihood) < tol:
            print("Converged!")
            break
        prev_likelihood = likelihood

    return pi, mu, sigma

# Generate synthetic data for testing
np.random.seed(42)
data1 = np.random.multivariate_normal([2, 2], [[1, 0.5], [0.5, 1]], size=100)
data2 = np.random.multivariate_normal([8, 8], [[1, -0.5], [-0.5, 1]], size=100)
data = np.vstack([data1, data2])

# Run the EM algorithm
k = 2  # Number of components
pi, mu, sigma = em_algorithm(data, k)

print("Final Parameters:")
print("Mixture Weights (pi):", pi)
print("Means (mu):", mu)
print("Covariances (sigma):", sigma)
